In [ ]:
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
dataset_path = "/content/drive/My Drive/Colab Notebooks/datasets/Flowers/Train"
width = height = 224
batch_size = 32

data_generator = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    # vertical_flip=True,
    brightness_range=(0.9, 1.1),
    shear_range=0.05,
    fill_mode='reflect',
    validation_split=0.1
    )

train_data = data_generator.flow_from_directory(
    dataset_path,
    target_size=(width, height),
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    subset='training'
    )

validation_data = data_generator.flow_from_directory(
    dataset_path,
    target_size=(width, height),
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    subset='validation'
    )

In [ ]:
train_images = next(train_data)[0]
plt.figure(figsize=(8,8)) # plot 25 images

for i in range(16):
    plt.subplot(4,4,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)

You have 4 options to prepare a model

1- DenseNet

In [ ]:
base_model = tf.keras.applications.DenseNet121(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(width, height, 3),
    pooling='avg'
)

In [ ]:
for layer in base_model.layers[:-4]:
    layer.trainable = False

In [ ]:
base_model.summary()

In [ ]:
model = tf.keras.Sequential([
    base_model,
    layers.Dense(train_data.num_classes, activation='softmax')
])

2- ResNet

In [ ]:
base_model = tf.keras.applications.ResNet152V2(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(width, height, 3),
    pooling='avg'
)

In [ ]:
for layer in base_model.layers[:-8]:
    layer.trainable = False

In [ ]:
base_model.summary()

In [ ]:
model = tf.keras.Sequential([
    base_model,
    # layers.Dropout(0.5),
    layers.Dense(train_data.num_classes, activation='softmax')
])

3- VGGNet

In [ ]:
base_model = tf.keras.applications.VGG16(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(width, height, 3),
    pooling='avg'
)

In [ ]:
for layer in base_model.layers[:-4]:
    layer.trainable = False

In [ ]:
base_model.summary()

In [ ]:
model = tf.keras.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(train_data.num_classes, activation='softmax')
])

4- PreTrained

In [ ]:
model = tf.keras.models.load_model('/content/drive/My Drive/Colab Notebooks/my_model')

In [ ]:
model.summary()

Train

In [ ]:
#@title Model compile
learning_rate = 2e-4 #@param {type:"slider", min:0.00001, max:0.001, step:0.00001}
model.compile(
    loss=tf.keras.losses.categorical_crossentropy,
    optimizer=tf.keras.optimizers.RMSprop(lr=learning_rate),
    # optimizer=tf.keras.optimizers.Adam(),
    # optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
    # optimizer=tf.keras.optimizers.Adadelta(learning_rate=0.0001),
    metrics='accuracy'
)

In [ ]:
#@title Model fit
epochs = 64 #@param {type:"slider", min:1, max:100, step:1}
history = model.fit(
    train_data,
    steps_per_epoch=train_data.samples // batch_size,
    validation_data=validation_data,
    validation_steps=validation_data.samples // batch_size,
    epochs=epochs,
    shuffle=True
)

In [ ]:
from matplotlib import pyplot as plt

plt.plot(history.history['loss'], label="Train")
plt.plot(history.history['val_loss'], label="Validation")
plt.legend(loc='best')
plt.xlabel('Epochs', fontsize=16)
plt.ylabel('Loss', fontsize=16)
plt.grid()
plt.show()

plt.plot(history.history['accuracy'], label="Train")
plt.plot(history.history['val_accuracy'], label="Validation")
plt.legend(loc='best')
plt.xlabel('Epochs', fontsize=16)
plt.ylabel('Accuracy', fontsize=16)
plt.grid()
plt.show()

Test

In [ ]:
dataset_path = "/content/drive/My Drive/Flowers/Test"
width = height = 224
batch_size = 32

data_generator = ImageDataGenerator(
    rescale=1./255,
    # rotation_range=8,
    # width_shift_range=0.1,
    # height_shift_range=0.1,
    horizontal_flip=True,
    # vertical_flip=True,
    # brightness_range=(0.8, 1.2),
    # shear_range=0.05,
    fill_mode='nearest')

test_data = data_generator.flow_from_directory(
        dataset_path,
        target_size=(width, height),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)

In [ ]:
model.evaluate(test_data)

In [ ]:
model.save('/content/drive/My Drive/Colab Notebooks/saved_model')

In [ ]:
#@title Download image
image_url = "https://img.freepik.com/free-photo/blooming-white-daisy-flower-isolated-black_112977-7.jpg" #@param {type:"string"}

import urllib.request
urllib.request.urlretrieve(image_url, 'flower.jpg')

In [ ]:
import cv2 as cv

img = cv.imread('flower.jpg')
img = cv.resize(img, (224, 224))
img = img / 255.0
img = img.reshape(1, 224, 224, 3)

output = model.predict(img)
print('probability', np.max(output))
print('class index', np.argmax(output))

labels = list(test_data.class_indices.keys())
print('class label', labels[np.argmax(output)])